In [2]:
import pandas as pd

In [97]:
all_features = ["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01",\
                    nrows = 10)
train['like_bool'] = train.like_timestamp.fillna(0)
train.loc[train.like_bool != 0.0, 'like_bool'] = 1.0
train = train.drop(['like_timestamp'], axis = 1)

In [48]:
train.dtypes

tweet_type                           object
Language                             object
enaged_with_user_id                  object
engaged_with_user_follower_count      int64
engaged_with_user_following_count     int64
engaged_with_user_is_verified          bool
engaging_user_id                     object
enaging_user_follower_count           int64
enaging_user_following_count          int64
enaging_user_is_verified               bool
engagee_follows_engager                bool
dtype: object

In [49]:
def bool_to_str(value):
    "value should be a bool"
    return 'Yes' if value else 'No'

In [226]:
def construct_line(row, columns, dtypes, string_dict, categorical_index):
#     print(string_dict)
    numerical_index = 0
    categorical_index = categorical_index
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        if ((dtypes[i] == object) or (dtypes[i] == bool)):
            if (dtypes[i] == bool):
                row[i] =  bool_to_str(row[i])
            values = string_dict[columns[i]]
            if row[i] in [x[0] for x in values]:
                index = [x[0] for x in values].index(row[i])
                value_index = values[index][1]
                new_line = new_line + str(value_index)+":1 "
            else:
                value_index = sum(len(v) for v in string_dict.values()) + categorical_index
                new_line = new_line + str(value_index)+":1 "
                values.append((row[i], value_index))
                string_dict[columns[i]] = values
        else:
            new_line = new_line + str(numerical_index) + ":" +str(row[i])+" "
            numerical_index = numerical_index + 1
    print(new_line)
    return string_dict

In [227]:
cnames_numerical = list(train.select_dtypes(exclude=['object', 'bool']).columns)
categorical_index = len(cnames_numerical)
string_dict = {k: [] for k in cnames_categorical}
train.apply(lambda x : construct_line(x, train.columns, train.dtypes, string_dict, categorical_index), axis = 1)

0.0 5:1 6:1 7:1 0:3517 1:590 8:1 9:1 2:27428 3:600 10:1 11:1 
0.0 5:1 12:1 13:1 0:39786 1:32978 8:1 14:1 2:17 3:77 10:1 15:1 
1.0 5:1 6:1 16:1 0:141726 1:3232 8:1 17:1 2:2 3:29 10:1 11:1 
0.0 5:1 6:1 18:1 0:3044 1:99 8:1 17:1 2:3 3:30 10:1 11:1 
0.0 5:1 6:1 16:1 0:141370 1:3235 8:1 17:1 2:3 3:30 10:1 11:1 
0.0 19:1 6:1 20:1 0:3264 1:39 8:1 17:1 2:3 3:29 10:1 11:1 
0.0 5:1 6:1 21:1 0:18133 1:177 8:1 22:1 2:94 3:650 10:1 11:1 
1.0 5:1 23:1 24:1 0:320 1:319 8:1 25:1 2:225 3:185 10:1 15:1 
0.0 5:1 23:1 26:1 0:6099673 1:755 27:1 25:1 2:222 3:184 10:1 11:1 
1.0 19:1 23:1 28:1 0:440 1:412 8:1 25:1 2:225 3:185 10:1 15:1 


0    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
1    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
2    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
3    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
4    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
5    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
6    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
7    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
8    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
9    {'tweet_type': [('TopLevel', 5), ('Retweet', 1...
dtype: object

In [228]:
train

,tweet_type,Language,enaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,engagee_follows_engager,like_bool
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,590,False,0000006C3074607050F1339DDCB890BB,27428,600,False,False,0.0
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,39786,32978,False,000005C520010F8917EEAB6F5B6EC1C4,17,77,False,True,0.0
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141726,3232,False,000006829BEADA9EEA695CF0C334B426,2,29,False,False,1.0
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,3044,99,False,000006829BEADA9EEA695CF0C334B426,3,30,False,False,0.0
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141370,3235,False,000006829BEADA9EEA695CF0C334B426,3,30,False,False,0.0
5,Retweet,22C448FF81263D4BAF2A176145EE9EAD,E4CF13FFB0C7B8AC747476D57543C413,3264,39,False,000006829BEADA9EEA695CF0C334B426,3,29,False,False,0.0
6,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D2D3B14476E8A5A33EC6435805B4B675,18133,177,False,00000776B07587ECA9717BFC301F2D6E,94,650,False,False,0.0
7,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,6DCC8D7CDE39AF4BA022CFE8F1436656,320,319,False,00000860E80C67D8C46CE57C64DE9444,225,185,False,True,1.0
8,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,FE036351A95ABC3827A499067C340755,6099673,755,True,00000860E80C67D8C46CE57C64DE9444,222,184,False,False,0.0
9,Retweet,ECED8A16BE2A5E8871FD55F4842F16B1,46E6134373F19D742F24D6E72FC74360,440,412,False,00000860E80C67D8C46CE57C64DE9444,225,185,False,True,1.0


In [81]:
cnames=list(train.select_dtypes(exclude=['object', 'bool']).columns)
len(cnames)

['engaged_with_user_follower_count',
 'engaged_with_user_following_count',
 'enaging_user_follower_count',
 'enaging_user_following_count']